In [1]:
__version__ = '3.1.6'
__author__ = 'SanoKei'

In [2]:
'''Imports'''

from tkinter import Tk, Frame, Canvas, PhotoImage
import tkinter as tk
import configparser
from multiprocessing import Process
import os
import random
import time
import json
import math
from PIL import Image, ImageTk
from threading import Thread
from selenium import webdriver
from win32api import GetMonitorInfo, MonitorFromPoint
from win32com.client import Dispatch

In [3]:
'''Initialize'''

# initialize tkinter
root = Tk()

# get the directory of the file
current_directory = os.getcwd()
davidImgPath = current_directory + '\\resources\\images\\David\\'

In [4]:
'''Get Stoff'''

# get taskbar height
def get_task_bar_height():
    monitor_info = GetMonitorInfo(MonitorFromPoint((0,0)))
    monitor_area = monitor_info.get("Monitor")
    work_area = monitor_info.get("Work")
    return monitor_area[3]-work_area[3]

barHeight = get_task_bar_height()

# get chrome version
def get_version_via_com(filename):
    parser = Dispatch("Scripting.FileSystemObject")
    try:
        version = parser.GetFileVersion(filename)
    except Exception:
        return None
    return version

paths = [r"C:\Program Files\Google\Chrome\Application\chrome.exe",r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"]
chrome_version = str(list(filter(None, [get_version_via_com(p) for p in paths]))[0]).split('.')[0]

# get direction to face david
def get_direction_for_david(action_rand, cords):
    fileName = "move"
    if (math.sqrt((root.winfo_rootx()-cords[0])**2) >= math.sqrt((root.winfo_rooty()-cords[1])**2)):
        fileName += ""
    else:
        fileName += "Y"
    if(cords[0] >= root.winfo_rootx()):
        # right
        fileName += "Right"
    else:
        fileName += "Left"
        
    return fileName
    
# get a random spot on the screen
def get_random_on_screen(width,height):
    x,y = random.choice(range(width)),random.choice(range(height))
    return [x,y]

In [5]:
'''David'''

### Images ###

# idle
idleLeft = ["DAVE_IDLE-LEFT_0_0.gif","DAVE_IDLE-LEFT_0_1.gif","DAVE_IDLE-LEFT_0_2.gif","DAVE_IDLE-LEFT_0_3.gif","DAVE_IDLE-LEFT_0_2.gif","DAVE_IDLE-LEFT_0_1.gif"]
idleRight = ["DAVE_IDLE-RIGHT_0_0.gif","DAVE_IDLE-RIGHT_0_1.gif","DAVE_IDLE-RIGHT_0_2.gif","DAVE_IDLE-RIGHT_0_3.gif","DAVE_IDLE-RIGHT_0_2.gif","DAVE_IDLE-RIGHT_0_1.gif"]
moveLeft = ["DAVE_MOVE-LEFT_0_0.gif","DAVE_MOVE-LEFT_0_1.gif"]
moveRight = ["DAVE_MOVE-RIGHT_0_0.gif","DAVE_MOVE-RIGHT_0_1.gif"]
moveYLeft = ["DAVE_MOVE_Y-LEFT_0_0.gif","DAVE_MOVE_Y-LEFT_0_1.gif","DAVE_MOVE_Y-LEFT_0_2.gif","DAVE_MOVE_Y-LEFT_0_3.gif","DAVE_MOVE_Y-LEFT_0_2.gif","DAVE_MOVE_Y-LEFT_0_1.gif",]
moveYRight = ["DAVE_MOVE_Y-RIGHT_0_0.gif","DAVE_MOVE_Y-RIGHT_0_1.gif","DAVE_MOVE_Y-RIGHT_0_2.gif","DAVE_MOVE_Y-RIGHT_0_3.gif","DAVE_MOVE_Y-RIGHT_0_2.gif","DAVE_MOVE_Y-RIGHT_0_1.gif",]

### Animation ###
animation = {
    "idleLeft": idleLeft, 
    "idleRight": idleRight, 
    "moveLeft": moveLeft, 
    "moveRight": moveRight,
    "moveYLeft": moveYLeft,
    "moveYRight": moveYRight,
    "webRight": idleRight
}

In [6]:
'''Chrome'''

### Chrome to Version ###
version_to_path = {
    "91": current_directory + "\\resources\\chromeDriver\\91_chromedriver.exe",
    "92": current_directory + "\\resources\\chromeDriver\\92_chromedriver.exe",
    "93": current_directory + "\\resources\\chromeDriver\\93_chromedriver.exe"
}

In [7]:
'''Find Config''' 

configParser = configparser.RawConfigParser()   
configFilePath = r''+os.getcwd()+'\\config.txt'
configParser.read(configFilePath)

['C:\\Users\\William\\Desktop\\David-The-Duck\\config.txt']

In [8]:
'''Read from config'''
### default ###
debug = bool(configParser.get('window', 'debug'))

### window ###
title = str(configParser.get('window', 'title'))
overrideredirect = bool(configParser.get('window', 'overrideredirect'))
top_most = bool(configParser.get('window', 'top_most'))
trans_color = str(configParser.get('window', 'trans_color'))

### david ###
speed = int(configParser.get('david', 'speed'))

### Actions and weights ###
actions = json.loads(configParser.get("david","actions"))
weights = json.loads(configParser.get("david","weights"))

### Files and animation ###
file_name = json.loads(configParser.get("david","file_name"))
rand = json.loads(configParser.get("david","rand"))
sleep = json.loads(configParser.get("david","sleep"))
minval = json.loads(configParser.get("david","minval"))
rick = json.loads(configParser.get("david","url"))

In [9]:
'''Configuration'''

### window ###
root.title(title)
root.overrideredirect(overrideredirect)
root.wm_attributes("-topmost", top_most)
root.wm_attributes("-disabled", True)
root.wm_attributes("-transparentcolor", trans_color)

''

In [10]:
'''David Class'''
class David(Frame):
    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.master = master
        self.pack(fill="both", expand=1)
        
    '''Dave Actions'''
    def dave_action(self,action,perFrame,photo,spd,url,cords,maxFrame,startPoint,perm_x,perm_y):
        '''
        var is perFrame
        '''
        ### move ###
        if("move" in action):
            x = cords[0]
            y = cords[1]
            srt_x = startPoint[0]
            srt_y = startPoint[1]
            if(x >= root.winfo_screenwidth()):
                x -= photo.width()
            
            if(y >= root.winfo_screenheight() - barHeight):
                y -= photo.height()
            
            # find angle and length of david to point
            
            # m is the slope and c is the point on the line
            m = (srt_y - cords[1]) / (srt_x - cords[0])
            dis = math.sqrt((srt_x-cords[0])**2 + (srt_y-cords[0])**2)
            incrim = dis / (maxFrame * spd)
            
            # this is getting the angle of slope 1 to slope 2
            angle = math.atan((1-m)/(1+m)
            if(debug):
                print("-"*20)
                print("frame: " + str(perFrame))
                print("Dis:" + str(dis))
                print("incrim:" + str(incrim))
            
            new_y = angle * (incrim * perFrame)
            new_x = math.sqrt((incrim * perFrame)**2 + new_y)
            perm_x += new_x
            perm_y += new_y
            
            if(debug):
                print("new_x:" + str(dis))
                print("new_y:" + str(incrim))
                print("perm_x:" + str(perm_x))
                print("perm_y:" + str(perm_y))
                print("-"*20)
            root.geometry("+{}+{}".format(str(int(math.ceil(perm_x))),str(int(math.ceil(perm_y)))))
            return [perm_x,perm_y]
            
        ### web ###
        if("webRight" in action):
            # create window
            if(var == 1):
                driver = webdriver.Chrome(executable_path=version_to_path.get(chrome_version))
                url = "https://www.youtube.com/watch?v=" + url
                driver.get(url)
                #window
            
            
            # spawn window randomly away from the screen and on the y access


            # move david to the right, move him outta frame

#             x = spd * var
#             if(x+root.winfo_rootx() >= root.winfo_screenwidth() - photo.width()):
#                 root.geometry("+{}+{}".format(str(root.winfo_screenwidth() - photo.width() - 1),str(root.winfo_rooty())))
#                 return True
#             else:
#                 root.geometry("+{}+{}".format(str(x+root.winfo_rootx()),str(root.winfo_rooty())))
#                 return False

            # when off screen, then bring onto screen
            

    '''Create the animation of David'''
    def createAnimation(self,imagelist,filePath,perFrame):
        path = davidImgPath+filePath+"\\"
        # extract width and height info
        photo = PhotoImage(file=path+imagelist[0])
        width = photo.width()
        height = photo.height()
        
        # create a list of image objects
        giflist = []
        for imagefile in imagelist:
            photo = PhotoImage(file=path+imagefile)
            giflist.append(photo)
            
        # loop through the gif image objects for a while
        gif = giflist[perFrame]
        canvas.delete("all")
        canvas.create_image(width/2.0, height/2.0, image=gif)
        canvas.update()
                
    def createAction(self,imagelist,filePath,perFrame,action,spd,url,cords,maxFrame,startPoint,perm_x,perm_y):
        path = davidImgPath+filePath+"\\"
        photo = PhotoImage(file=path+imagelist[0])
        return self.dave_action(action,perFrame,photo,spd,url,cords,maxFrame,startPoint,perm_x,perm_y)

In [ ]:
'''Run main window'''

# initialize class of David
david = David(root)

# Create the canvas
canvas = Canvas(width=128, height=128, bg=trans_color, highlightthickness=0)
canvas.pack()

while True:
    # random action
    action_rand = random.choices(actions, weights=weights)
    index = actions.index(str(action_rand[0]))
    t = random.choice(range(list(rand)[index])) + int(minval[index])
    slp = list(sleep)[index]
    spd = random.choice(range(speed)) + int(minval[index])
    roll = random.choice(rick)
    ranSoFarAway = get_random_on_screen(root.winfo_screenwidth(),root.winfo_screenheight() - barHeight)
    startPoint = [root.winfo_rootx(),root.winfo_rooty()]
    perm_x = perm_y = 0
    perFrame = 0
    # sterialize animation
    if(action_rand[0] == "move"):
        action_rand = get_direction_for_david(action_rand,ranSoFarAway)
    if isinstance(action_rand, list):
        act_rand = action_rand[0]
    else:
        act_rand = action_rand
    maxFrame = t * len(animation.get(act_rand))
    
    print("H:"+str(root.winfo_screenheight()) +"W:"+str(root.winfo_screenwidth()) )
    
    # animation and action main loop
    for perAnimationLoop in range(t):
        for perFrameAnimation in range(len(animation.get(act_rand))):
            perFrame += 1
            david.createAnimation(animation.get(act_rand),file_name[index],perFrameAnimation)
            [perm_x,perm_y] = david.createAction(animation.get(act_rand),file_name[index],perFrame,action_rand,spd,roll,ranSoFarAway,maxFrame,startPoint,perm_x,perm_y)  
            time.sleep(slp)
    
root.mainloop()

H:1080W:1920
--------------------
frame: 1
Dis:1453.8115421195416
incrim:22.715805345617838
new_x:1453.8115421195416
new_y:22.715805345617838
perm_x:22.95578287944873
perm_y:10.960155108391486
--------------------
--------------------
frame: 2
Dis:1453.8115421195416
incrim:22.715805345617838
new_x:1453.8115421195416
new_y:22.715805345617838
perm_x:68.62800157108646
perm_y:32.880465325174455
--------------------
--------------------
frame: 3
Dis:1453.8115421195416
incrim:22.715805345617838
new_x:1453.8115421195416
new_y:22.715805345617838
perm_x:137.01623723940278
perm_y:65.76093065034891
--------------------
--------------------
frame: 4
Dis:1453.8115421195416
incrim:22.715805345617838
new_x:1453.8115421195416
new_y:22.715805345617838
perm_x:228.12038434826695
perm_y:109.60155108391486
--------------------
--------------------
frame: 5
Dis:1453.8115421195416
incrim:22.715805345617838
new_x:1453.8115421195416
new_y:22.715805345617838
perm_x:341.94040054946925
perm_y:164.4023266258723
--

--------------------
frame: 10
Dis:867.3292339129357
incrim:4.015413119967295
new_x:867.3292339129357
new_y:4.015413119967295
perm_x:261.6422196305153
perm_y:2035.9399334834177
--------------------
--------------------
frame: 11
Dis:867.3292339129357
incrim:4.015413119967295
new_x:867.3292339129357
new_y:4.015413119967295
perm_x:310.2028686387539
perm_y:2443.1279201801012
--------------------
--------------------
frame: 12
Dis:867.3292339129357
incrim:4.015413119967295
new_x:867.3292339129357
new_y:4.015413119967295
perm_x:362.7955981729126
perm_y:2887.3329965764833
--------------------
--------------------
frame: 13
Dis:867.3292339129357
incrim:4.015413119967295
new_x:867.3292339129357
new_y:4.015413119967295
perm_x:419.4180394188798
perm_y:3368.5551626725637
--------------------
--------------------
frame: 14
Dis:867.3292339129357
incrim:4.015413119967295
new_x:867.3292339129357
new_y:4.015413119967295
perm_x:480.06829572914495
perm_y:3886.794418468343
--------------------
----------

--------------------
frame: 15
Dis:21639.251304053938
incrim:33.393906333416574
new_x:21639.251304053938
new_y:33.393906333416574
perm_x:3757.436815120822
perm_y:-123719.91027526335
--------------------
--------------------
frame: 16
Dis:21639.251304053938
incrim:33.393906333416574
new_x:21639.251304053938
new_y:33.393906333416574
perm_x:4276.072694033743
perm_y:-140215.89831196514
--------------------
--------------------
frame: 17
Dis:21639.251304053938
incrim:33.393906333416574
new_x:21639.251304053938
new_y:33.393906333416574
perm_x:4828.116373456168
perm_y:-157742.88560096078
--------------------
--------------------
frame: 18
Dis:21639.251304053938
incrim:33.393906333416574
new_x:21639.251304053938
new_y:33.393906333416574
perm_x:5413.56626802028
perm_y:-176300.8721422503
--------------------
--------------------
frame: 19
Dis:21639.251304053938
incrim:33.393906333416574
new_x:21639.251304053938
new_y:33.393906333416574
perm_x:6032.421049096117
perm_y:-195889.85793583366
--------